In [ ]:
# MAX_ATTEMPTS = 1
# RUN_ID = 8
# RUN_NOTE = "Second run with code agent only"
# RUN_AGENTS = ['code', 'github', 'docs', 'git']

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import os
import sys
import time
BASE_DIR = os.path.abspath(os.path.abspath("../"))
sys.path.append(BASE_DIR)

In [ ]:
import pandas as pd
from langchain_core.messages import BaseMessage, HumanMessage
from src.orchestration.graph import graph
from src.utils import safe_get_content, remove_think_block

In [ ]:
questions = pd.read_csv(BASE_DIR+"/data/external/final_questions.csv", index_col='id')
questions

In [ ]:
projects = pd.read_csv(BASE_DIR+"/data/processed/projects.csv")
projects["repo_path"] = projects["repo"].apply(lambda x: f"{BASE_DIR}/data/processed/{x.split('/')[-1]}")

In [ ]:
projects.head(5)

In [ ]:
data = pd.merge(questions, projects, left_on='project', right_on='repo', how='inner')
data.head(5)

In [ ]:
runs = pd.read_csv("../data/external/runs.csv")
runs

In [ ]:
for _, row in runs.iterrows():
    RUN_ID = row['run_id']
    RUN_NOTE = row['note']
    RUN_GIT = row['git']
    RUN_GITHUB = row['github']
    RUN_CODE = row['code']
    RUN_DOCS = row['docs']
    start = time.time()
    generated = []
    for i, row in data.iterrows():
        question = row['customized_quesstion']
        # result = graph.invoke
        code_db = BASE_DIR+"/data/processed/"+row['code_db']
        repository_path = row['repo_path']
        github_id = row['repo']
        docs_url = row['docs_url']
        attempt = 0
            # response = f"Response for question '{question}' on {repository_path} with code_db {code_db} and github_id {github_id} running agents: {RUN_AGENTS}"
        try: 
            print(f"Running question {RUN_ID} ")
            result = graph.invoke({
                "user_query": [HumanMessage(question)],
                "source_db": code_db,
                "repository_path": repository_path,
                "docs_source": docs_url,
                "github_url": github_id,
                "run_git": RUN_GIT,
                "run_github": RUN_GITHUB,
                "run_code":RUN_CODE,
                "run_docs": RUN_DOCS
                })
            if(result["final_response"]):
                final_response = result['final_response'].content
            else:
                final_response = 'Error: cound not answer'
            
        except Exception as e:
            print(f"Error: {e}")
            final_response = "Error: cound not answer"
        
        generated.append({
            "run_id": RUN_ID,
            "question_id": row["question_id"],
            "question": question,
            "project": github_id,
            "final_response": final_response,
        })  
        generated_df = pd.DataFrame(generated)
        generated_df.to_csv(f"{BASE_DIR}/data/generated/generated_{RUN_ID}.csv", index=False)
    end = time.time()
    with open(f"{BASE_DIR}/data/generated/completed_runs.csv", "a") as f:
        f.write(f"{RUN_ID},{len(generated_df)},{end - start:.4f}\n")

In [ ]:
generated_df = pd.DataFrame(generated)

In [ ]:
generated_df.head()

In [ ]:
generated_df.to_csv(f"{BASE_DIR}/data/generated/generated_{RUN_ID}.csv", index=False)

In [ ]:

# Get current formatted time
